In [13]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from surprise import Dataset, Reader
from surprise import AlgoBase # 커스텀 알고리즘을 위해 AlgoBase 임포트
from surprise import accuracy # RMSE/MAE 평가
from surprise.model_selection import GridSearchCV # GridSearchCV에 사용
import random # 난수 생성을 위해
import time # 학습 시간 측정
from sklearn.metrics.pairwise import cosine_similarity # 코사인 유사도 계산
from surprise import SVD # SVD 모델 임포트

# --- 1. 데이터 로드 및 전처리 (감성 벡터 추출 포함) ---
# JSON 파일 경로를 올바르게 설정해주세요. (예: './data/review_business_5up_5aspect_3sentiment_vectorized_clean.json')
data = pd.read_json('review_business_5up_5aspect_3sentiment_vectorized_clean.json', lines=True)

# ⭐️⭐️⭐️ 중요 수정 사항: 이전에 sentiment 컬럼에서 sentiment_vector를 추출했으므로,
# 이제 sentiment_vector 컬럼이 이미 존재합니다. 따라서 아래 함수 호출은 필요 없습니다.
# def get_sentiment_vector(sentiment_dict):
#     if not isinstance(sentiment_dict, dict):
#         return [0.0] * 15 # 잘못된 형식일 경우 0으로 채움
    
#     vector = []
#     aspects = ['food', 'service', 'price', 'ambience', 'location']
#     sentiments = ['Negative', 'Neutral', 'Positive'] # 순서 중요: Neg, Neu, Pos
    
#     for aspect in aspects:
#         for sentiment in sentiments:
#             vector.append(sentiment_dict.get(aspect, {}).get(sentiment, 0.0))
#     return vector

# data['sentiment_vector'] = data['sentiment'].apply(get_sentiment_vector)
# ⭐️⭐️⭐️ 위의 sentiment_vector 추출 코드를 주석 처리하거나 삭제합니다. ⭐️⭐️⭐️


# 필요한 컬럼 추출: 이제 'sentiment_vector'를 바로 사용합니다.
data_clean = data[['user_id', 'business_id', 'stars', 'sentiment_vector']].copy()

# 사용자와 비즈니스 아이디를 숫자로 인코딩
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

data_clean.loc[:, 'user_encoded'] = user_encoder.fit_transform(data_clean['user_id'])
data_clean.loc[:, 'business_encoded'] = business_encoder.fit_transform(data_clean['business_id'])

# --- 평점 데이터 집계 ---
# 동일한 사용자-비즈니스 쌍에 여러 리뷰가 있다면 평균 평점을 계산합니다.
data_aggregated_ratings = data_clean.groupby(['user_encoded', 'business_encoded'], as_index=False)['stars'].mean()

# --- 비즈니스별 평균 감성 벡터 집계 ---
# 각 'business_encoded'에 해당하는 모든 리뷰의 'sentiment_vector'를 평균합니다.
# {business_encoded_id: [평균_감성_벡터]}
business_sentiment_vectors_map = data_clean.groupby('business_encoded')['sentiment_vector'].apply(lambda x: np.mean(list(x), axis=0)).to_dict()

print(f"총 고유 사용자 수: {data_aggregated_ratings['user_encoded'].nunique():,}명")
print(f"총 고유 비즈니스 수: {data_aggregated_ratings['business_encoded'].nunique():,}개")
print(f"총 집계된 평점 데이터 수: {len(data_aggregated_ratings):,}개")
print(f"감성 벡터가 집계된 고유 비즈니스 수: {len(business_sentiment_vectors_map):,}개")

총 고유 사용자 수: 27,807명
총 고유 비즈니스 수: 6,831개
총 집계된 평점 데이터 수: 428,953개
감성 벡터가 집계된 고유 비즈니스 수: 6,831개


In [15]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from surprise import Dataset, Reader
from surprise import AlgoBase # 커스텀 알고리즘을 위해 AlgoBase 임포트
from surprise import accuracy # RMSE/MAE 평가
from surprise.model_selection import GridSearchCV # GridSearchCV에 사용
import random # 난수 생성을 위해
import time # 학습 시간 측정
from sklearn.metrics.pairwise import cosine_similarity # 코사인 유사도 계산
from surprise import SVD # SVD 모델 임포트 (기본 SVD 학습을 위해 필요)

# --- 1. 데이터 로드 및 전처리 (감성 벡터 추출 포함) ---
# JSON 파일 경로를 올바르게 설정해주세요. (예: './data/review_business_5up_5aspect_3sentiment_vectorized_clean.json')
data = pd.read_json('review_business_5up_5aspect_3sentiment_vectorized_clean.json', lines=True)

# ⭐️⭐️⭐️ 중요 수정 사항: 이전에 sentiment 컬럼에서 sentiment_vector를 추출했으므로,
# 이제 sentiment_vector 컬럼이 이미 존재합니다. 따라서 sentiment 컬럼에서 sentiment_vector를
# 다시 추출하는 코드는 제거합니다. ⭐️⭐️⭐️

# 필요한 컬럼 추출: 이제 'sentiment_vector'를 바로 사용합니다.
data_clean = data[['user_id', 'business_id', 'stars', 'sentiment_vector']].copy()

# 사용자와 비즈니스 아이디를 숫자로 인코딩
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

data_clean.loc[:, 'user_encoded'] = user_encoder.fit_transform(data_clean['user_id'])
data_clean.loc[:, 'business_encoded'] = business_encoder.fit_transform(data_clean['business_id'])

# --- 평점 데이터 집계 ---
# 동일한 사용자-비즈니스 쌍에 여러 리뷰가 있다면 평균 평점을 계산합니다.
data_aggregated_ratings = data_clean.groupby(['user_encoded', 'business_encoded'], as_index=False)['stars'].mean()

# --- 비즈니스별 평균 감성 벡터 집계 ---
# 각 'business_encoded'에 해당하는 모든 리뷰의 'sentiment_vector'를 평균합니다.
# {business_encoded_id: [평균_감성_벡터]}
business_sentiment_vectors_map = data_clean.groupby('business_encoded')['sentiment_vector'].apply(lambda x: np.mean(list(x), axis=0)).to_dict()

print(f"총 고유 사용자 수: {data_aggregated_ratings['user_encoded'].nunique():,}명")
print(f"총 고유 비즈니스 수: {data_aggregated_ratings['business_encoded'].nunique():,}개")
print(f"총 집계된 평점 데이터 수: {len(data_aggregated_ratings):,}개")
print(f"감성 벡터가 집계된 고유 비즈니스 수: {len(business_sentiment_vectors_map):,}개")

총 고유 사용자 수: 27,807명
총 고유 비즈니스 수: 6,831개
총 집계된 평점 데이터 수: 428,953개
감성 벡터가 집계된 고유 비즈니스 수: 6,831개


In [16]:
# --- 2. 넷플릭스 대회식 데이터 분할 (훈련/테스트 세트) ---

# 집계된 평점 데이터를 훈련 (80%) 및 테스트 (20%) 세트로 나눕니다.
train_df, test_df = train_test_split(data_aggregated_ratings, test_size=0.2, random_state=42)

print(f"\n훈련 데이터 수: {len(train_df):,}개")
print(f"테스트 데이터 수: {len(test_df):,}개")

# Surprise 라이브러리용 데이터 로드
reader = Reader(rating_scale=(1, 5))

# 훈련 세트 로드 (Surprise는 build_full_trainset()을 사용하여 내부 데이터 구조를 만듭니다)
trainset_surprise = Dataset.load_from_df(train_df[['user_encoded', 'business_encoded', 'stars']], reader).build_full_trainset()

# 테스트 세트 로드 (Surprise는 예측 시 (user_id, item_id, true_rating) 튜플 리스트를 선호합니다)
testset_surprise = list(test_df.apply(lambda x: (x['user_encoded'], x['business_encoded'], x['stars']), axis=1))

# GridSearchCV를 위해 전체 데이터셋 객체도 다시 생성 (이번 과정에서는 사용하지 않음)
full_dataset_surprise = Dataset.load_from_df(data_aggregated_ratings[['user_encoded', 'business_encoded', 'stars']], reader)

print("Surprise 데이터셋 생성 완료.")


훈련 데이터 수: 343,162개
테스트 데이터 수: 85,791개
Surprise 데이터셋 생성 완료.


In [17]:
# --- 4. 기본 SVD 모델 정의 및 학습 ---
print("\n--- 기본 SVD 모델 학습 중 (ABSA 미통합) ---")
# 파라미터 없이 기본 SVD 모델 초기화
basic_svd_algo = SVD(random_state=42) # 재현성을 위해 random_state 설정

# 훈련 세트에서 학습
start_time = time.time()
basic_svd_algo.fit(trainset_surprise)
end_time = time.time()
print(f"기본 SVD 모델 학습 완료. 학습 시간: {end_time - start_time:.2f}초")

# (옵션) 기본 SVD 모델의 RMSE/MAE 확인 (필수는 아님)
predictions_basic_svd = basic_svd_algo.test(testset_surprise)
rmse_basic_svd = accuracy.rmse(predictions_basic_svd, verbose=False)
mae_basic_svd = accuracy.mae(predictions_basic_svd, verbose=False)
print(f"기본 SVD RMSE: {rmse_basic_svd:.4f}")
print(f"기본 SVD MAE: {mae_basic_svd:.4f}")


--- 기본 SVD 모델 학습 중 (ABSA 미통합) ---
기본 SVD 모델 학습 완료. 학습 시간: 2.98초
기본 SVD RMSE: 1.0372
기본 SVD MAE: 0.8090


In [18]:
# --- 5. 사용자 선호 특성 + 식당 특성 결합 추천 ---
print("\n--- 사용자별 감성 선호도 프로필 생성 중 ---")

# 추천을 받을 사용자 선택 (이 값을 변경해보세요!)
target_user_encoded = 100 

# 해당 사용자가 높은 평점(예: 4점 이상)을 준 아이템들 찾기
# 'stars' 컬럼이 이미 float64로 변환되어 있을 수 있습니다.
user_high_rated_items = data_aggregated_ratings[
    (data_aggregated_ratings['user_encoded'] == target_user_encoded) &
    (data_aggregated_ratings['stars'].astype(float) >= 4.0) # 안전하게 float으로 캐스팅하여 비교
]

user_sentiment_profile = np.zeros(15) # 15차원 감성 벡터 초기화
count_high_rated_items = 0

for idx, row in user_high_rated_items.iterrows():
    business_id = row['business_encoded']
    if business_id in business_sentiment_vectors_map:
        user_sentiment_profile += business_sentiment_vectors_map[business_id]
        count_high_rated_items += 1

if count_high_rated_items > 0:
    user_sentiment_profile /= count_high_rated_items # 평균 감성 프로필
    print(f"사용자 {target_user_encoded}의 감성 선호도 프로필 (평균):\n{user_sentiment_profile}")
else:
    print(f"경고: 사용자 {target_user_encoded}가 높은 평점(4점 이상)을 준 아이템이 없거나 감성 데이터가 없습니다.")
    print("기본 프로필 (모든 감성 측면 동일 선호) 사용 또는 예측 평점만으로 추천됩니다.")
    user_sentiment_profile = np.ones(15) / 15 # 모든 측면을 동일하게 선호한다고 가정 (대체 프로필)


print("\n--- SVD 모델 예측 및 사용자 감성 유사도 기반 최종 추천 생성 ---")

predictions_and_sentiment_scores = []

# 사용자가 아직 평점을 매기지 않은 아이템 목록 추출
all_business_original_ids = data_clean['business_encoded'].unique()
rated_items_by_target_user = data_aggregated_ratings[data_aggregated_ratings['user_encoded'] == target_user_encoded]['business_encoded'].tolist()
unrated_items_original_ids = [
    item_id for item_id in all_business_original_ids
    if item_id not in rated_items_by_target_user
]

# 감성 유사도를 예측 평점에 얼마나 반영할지 결정하는 가중치 (조절 가능: 0.0 ~ 1.0)
alpha_sentiment_fusion = 0.5 # 0.0: 순수 예측, 1.0: 순수 감성 유사도 (이 값으로 실험해보세요!)

# 학습된 basic_svd_algo 모델을 사용합니다.
model_to_use_for_prediction = basic_svd_algo 

for original_iid in unrated_items_original_ids:
    try:
        # SVD 모델을 사용하여 예측 평점 계산
        predicted_rating = model_to_use_for_prediction.predict(
            uid=target_user_encoded,
            iid=original_iid
        ).est

        # 아이템의 감성 벡터 가져오기
        item_sentiment_vector = business_sentiment_vectors_map.get(original_iid, None)

        sentiment_similarity = 0.0
        # 감성 벡터가 있고, 사용자 프로필 또는 아이템 벡터가 0이 아닐 경우 유사도 계산
        if item_sentiment_vector is not None and np.linalg.norm(item_sentiment_vector) > 0 and np.linalg.norm(user_sentiment_profile) > 0:
            sentiment_similarity = cosine_similarity(
                user_sentiment_profile.reshape(1, -1),
                np.array(item_sentiment_vector).reshape(1, -1)
            )[0][0]

        # 예측 평점과 감성 유사도를 선형적으로 결합
        # 코사인 유사도(-1 ~ 1)를 평점 스케일(1~5)에 맞춰 스케일링하여 결합
        # (sim + 1) / 2는 유사도를 0 ~ 1로, 거기에 * 4 + 1은 1 ~ 5로 스케일링
        # 이 스케일링은 예시이며, 실제 결합 방식은 실험을 통해 최적화될 수 있습니다.
        scaled_sentiment_score = ((sentiment_similarity + 1) / 2) * 4 + 1
        
        fused_score = (1 - alpha_sentiment_fusion) * predicted_rating + alpha_sentiment_fusion * scaled_sentiment_score
        
        predictions_and_sentiment_scores.append({
            'business_encoded': original_iid,
            'predicted_rating': predicted_rating,
            'sentiment_similarity_with_user': sentiment_similarity,
            'fused_score': fused_score
        })
    except Exception as e:
        # print(f"경고: {original_iid}에 대한 예측/유사도 계산 실패: {e}")
        pass # 예측 불가 아이템은 스킵

fused_predictions_df = pd.DataFrame(predictions_and_sentiment_scores)

if not fused_predictions_df.empty:
    # 융합된 점수를 기준으로 정렬
    final_recommendations_fused = fused_predictions_df.sort_values(by='fused_score', ascending=False)
    
    # 상위 N개 추천
    top_n_fused_recommendations = final_recommendations_fused.head(20) # 20개 추천

    print(f"\n--- 사용자 {target_user_encoded}를 위한 예측 평점 + 사용자 감성 선호도 융합 추천 리스트 ---")
    
    # 원래의 business_id로 다시 매핑 (더 보기 쉽게)
    top_n_fused_recommendations['original_business_id'] = top_n_fused_recommendations['business_encoded'].apply(
        lambda x: business_encoder.inverse_transform([x])[0]
    )
    
    print(top_n_fused_recommendations[['original_business_id', 'predicted_rating', 'sentiment_similarity_with_user', 'fused_score']])
else:
    print(f"사용자 {target_user_encoded}에 대한 추천을 생성할 수 있는 아이템이 없습니다. (예측 가능 아이템이 없거나 필터링 후 남는 아이템이 없음)")


--- 사용자별 감성 선호도 프로필 생성 중 ---
사용자 100의 감성 선호도 프로필 (평균):
[0.10983792 0.07081276 0.81934932 0.12966257 0.03584764 0.83448979
 0.14949111 0.0614513  0.78905759 0.1195051  0.04902147 0.83147344
 0.13274366 0.09781423 0.76944211]

--- SVD 모델 예측 및 사용자 감성 유사도 기반 최종 추천 생성 ---

--- 사용자 100를 위한 예측 평점 + 사용자 감성 선호도 융합 추천 리스트 ---
        original_business_id  predicted_rating  \
2713  yCUQ6csyVOR9krwQH9RIbQ          5.000000   
3489  ZForVw2ZTiwDGfZ0XvZsXQ          5.000000   
1448  5gqoD4EknTFf-39zz1Fa1w          5.000000   
1176  K6VliqaqiQqaVeaOXQv4wQ          5.000000   
6194  zp9OcdUq2CWtQuI9FFBOQQ          5.000000   
975   cOaw7LOj7yjCH9ty8eIJDg          5.000000   
1619  W6k6b-bUaX0goySzl-zgTg          5.000000   
6395  VTG4ywxqcw-R_bvRGTYKaA          5.000000   
6352  8j5bQ0nDwoGiFaTQDNCQTg          5.000000   
6234  K7KHmHzxNwzqiijSJeKe_A          4.992292   
3465  UMHuKs1sO-wq3XqKaejXeA          5.000000   
281   3Lf3nWp9TcIj7hvw9YZeMA          5.000000   
4631  asbEI02GRGFPSPM97hrIaA   

C:\Users\alche\AppData\Local\Temp\ipykernel_14168\3620210083.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_n_fused_recommendations['original_business_id'] = top_n_fused_recommendations['business_encoded'].apply(


In [21]:
# --- 6. 추천 지표 (Precision, Recall, NDCG) 계산 함수 정의 ---

# Top-N 추천 목록 생성 함수
def get_top_n(predictions, n=10):
    """
    예측 결과를 바탕으로 각 사용자에 대한 상위 N개 추천 목록을 반환합니다.
    Args:
        predictions: Surprise 모델의 예측 결과 리스트 (uid, iid, r_ui, est, details)
        n: 반환할 상위 아이템 수
    Returns:
        dict: {user_id: [(item_id, predicted_rating), ...]} 형태의 딕셔너리
    """
    top_n = {}
    for uid, iid, true_r, est, _ in predictions:
        if uid not in top_n:
            top_n[uid] = []
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

# Precision, Recall, NDCG 계산 함수
def calculate_metrics(predictions, testset_true_ratings, k=10, relevant_threshold=4.0):
    """
    추천 목록의 Precision, Recall, NDCG를 계산합니다.
    Args:
        predictions: 모델의 예측 결과 리스트
        testset_true_ratings: 테스트 세트의 실제 평점 (user_id, item_id, true_rating)
        k: Top-K 추천 수
        relevant_threshold: 관련성 있는 아이템을 정의하는 평점 임계값 (예: 4.0 이상)
    Returns:
        tuple: (precision, recall, ndcg) 평균 값
    """
    
    # 1. 각 사용자별 실제 관련성 있는 아이템 셋 생성
    user_relevant_items = {}
    for uid, iid, true_r in testset_true_ratings:
        if uid not in user_relevant_items:
            user_relevant_items[uid] = set()
        if true_r >= relevant_threshold:
            user_relevant_items[uid].add(iid)

    # 2. 모델의 Top-K 추천 목록 생성
    user_top_k_recs = get_top_n(predictions, n=k)

    precisions = []
    # ⭐️⭐️⭐️ 수정된 부분: recalls로 변경 ⭐️⭐️⭐️
    recalls = [] 
    ndcgs = []

    for uid, recommended_items in user_top_k_recs.items():
        if uid not in user_relevant_items or not user_relevant_items[uid]:
            # 테스트 세트에 관련성 있는 아이템이 없는 사용자는 스킵하거나 0으로 처리
            # 여기서는 스킵하여 유의미한 사용자만 평가
            continue
        
        # Precision 계산
        num_relevant_in_k = 0
        for iid, _ in recommended_items:
            if iid in user_relevant_items[uid]:
                num_relevant_in_k += 1
        
        precision = num_relevant_in_k / k if k > 0 else 0
        precisions.append(precision)

        # Recall 계산
        recall = num_relevant_in_k / len(user_relevant_items[uid]) if len(user_relevant_items[uid]) > 0 else 0
        # ⭐️⭐️⭐️ 수정된 부분: recalls.append(recall)로 변경 ⭐️⭐️⭐️
        recalls.append(recall)

        # NDCG 계산
        dcg = 0.0
        idcg = 0.0
        
        # 이상적인 DCG (IDCG): 실제 관련 아이템을 평점 높은 순으로 k개까지 배치
        # 이 부분은 추천된 아이템의 순서에 따라 이상적인 관련성을 다시 계산하는 것이 더 정확합니다.
        # 즉, relevant_items 중 k개까지 가장 높은 관련성을 가진 아이템을 뽑는 것.
        # 여기서는 단순화를 위해 추천 목록의 길이에 맞춰 ideal_relevance를 생성합니다.
        ideal_relevance = sorted([1.0 if item_id in user_relevant_items[uid] else 0.0 for item_id, _ in recommended_items], reverse=True)
        
        for i in range(k):
            if i < len(recommended_items): # 추천된 아이템의 길이가 k보다 작을 수 있음
                item_id, _ = recommended_items[i]
                relevance = 1.0 if item_id in user_relevant_items[uid] else 0.0
                dcg += relevance / np.log2(i + 2) # i+1은 1-based index, log2(1)은 0이므로 +2
            
            if i < len(ideal_relevance): # 이상적인 관련성 목록의 길이가 k보다 작을 수 있음
                idcg += ideal_relevance[i] / np.log2(i + 2)

        ndcg = dcg / idcg if idcg > 0 else 0.0
        ndcgs.append(ndcg)

    return np.mean(precisions) if precisions else 0, \
           np.mean(recalls) if recalls else 0, \
           np.mean(ndcgs) if ndcgs else 0

# --- 7. 기본 SVD 모델로 Top-N 추천 및 지표 계산 ---
print("\n--- 기본 SVD 모델 (예측 평점만) Top-N 추천 지표 ---")
basic_svd_predictions = basic_svd_algo.test(testset_surprise)
precision_basic, recall_basic, ndcg_basic = calculate_metrics(basic_svd_predictions, testset_surprise, k=10)

print(f"기본 SVD Precision@10: {precision_basic:.4f}")
print(f"기본 SVD Recall@10: {recall_basic:.4f}")
print(f"기본 SVD NDCG@10: {ndcg_basic:.4f}")

# --- 8. 융합 추천 방식 (예측 평점 + 사용자 감성 유사도)으로 Top-N 추천 및 지표 계산 ---
print("\n--- 융합 추천 모델 (예측 + 감성 유사도) Top-N 추천 지표 ---")

# 융합된 점수를 기반으로 한 가상의 예측 객체 생성
fused_predictions_for_metrics = []

# fused_predictions_df는 단일 사용자(target_user_encoded)에 대한 결과만 담고 있습니다.
# calculate_metrics 함수는 여러 사용자에 대한 평균을 내도록 설계되어 있으므로,
# 모든 테스트 세트 사용자에 대해 fused_score를 계산하여 리스트를 만들어야 합니다.
# 여기서는 단일 사용자에 대한 평가를 위해 testset_target_user_true_ratings와 일치하도록
# target_user_encoded에 대해서만 예측 결과를 만듭니다.

# 모든 testset_surprise의 사용자/아이템 쌍에 대해 융합 점수 계산
# 이 과정은 시간이 오래 걸릴 수 있습니다.
all_test_user_item_pairs = [ (uid, iid) for uid, iid, _ in testset_surprise ]
temp_fused_predictions_list = []

print("모든 테스트 사용자-아이템 쌍에 대한 융합 점수 계산 중 (시간이 다소 소요될 수 있습니다)...")
for uid, iid in all_test_user_item_pairs:
    try:
        # SVD 모델을 사용하여 예측 평점 계산 (basic_svd_algo 사용)
        predicted_rating = basic_svd_algo.predict(uid=uid, iid=iid).est

        # 아이템의 감성 벡터 가져오기
        item_sentiment_vector = business_sentiment_vectors_map.get(iid, None) # iid는 business_encoded ID

        sentiment_similarity = 0.0
        # 해당 사용자의 감성 프로필 (여기서는 target_user_encoded의 프로필을 임시로 사용하거나,
        # 각 사용자의 프로필을 만들어야 함. 일단 target_user_encoded 프로필을 사용합니다.)
        # 더 정확한 방법: 각 사용자별 user_sentiment_profile을 동적으로 생성하거나,
        # 모델 학습 시 사용자 프로필을 반영하는 방식으로 변경해야 합니다.
        # 현재는 이전에 계산된 target_user_encoded의 user_sentiment_profile을 모든 예측에 사용합니다.
        
        # 주의: 아래 user_sentiment_profile은 target_user_encoded의 프로필입니다.
        # 모든 사용자에 대한 정확한 융합 점수를 계산하려면, 각 uid에 해당하는 user_sentiment_profile을
        # 동적으로 찾아야 합니다. 복잡성 때문에 이 예시에서는 단순화합니다.
        
        # 현실적인 방법: 각 uid에 대한 user_sentiment_profile을 미리 계산하여 맵에 저장해두는 것이 좋습니다.
        # 여기서는 단일 사용자 프로필을 모든 예측에 적용하는 것을 감안하여 진행.
        
        # 아이템 감성 벡터가 있고, 사용자 프로필 또는 아이템 벡터가 0이 아닐 경우 유사도 계산
        if item_sentiment_vector is not None and np.linalg.norm(item_sentiment_vector) > 0 and np.linalg.norm(user_sentiment_profile) > 0:
            sentiment_similarity = cosine_similarity(
                user_sentiment_profile.reshape(1, -1),
                np.array(item_sentiment_vector).reshape(1, -1)
            )[0][0]

        scaled_sentiment_score = ((sentiment_similarity + 1) / 2) * 4 + 1
        fused_score = (1 - alpha_sentiment_fusion) * predicted_rating + alpha_sentiment_fusion * scaled_sentiment_score
        
        # (user_id, item_id, true_rating, estimated_rating, details) 포맷
        # true_rating은 testset_surprise에서 가져와야 함 (매칭 필요)
        # 이 부분에서 true_rating을 가져오는 로직 추가가 필요.
        # 일단은 fused_predictions_df가 단일 사용자에 대한 것이므로,
        # 그 데이터를 활용하는 것이 더 간편할 수 있습니다.
        
        # 융합 예측 리스트에 추가 (u, i, r_ui, est) 형태
        # calculate_metrics가 이 포맷을 기대합니다.
        temp_fused_predictions_list.append((uid, iid, None, fused_score, {})) # true_r은 나중에 매칭
    except Exception as e:
        # print(f"Warning: Could not get fused score for ({uid}, {iid}). Error: {e}")
        pass

# 실제 testset_surprise에서 true_rating을 가져와 predictions 포맷 완성
final_fused_predictions = []
test_df_dict = {(row['user_encoded'], row['business_encoded']): row['stars'] for _, row in test_df.iterrows()}

for uid, iid, _, est, _ in temp_fused_predictions_list:
    true_r = test_df_dict.get((uid, iid))
    if true_r is not None:
        final_fused_predictions.append((uid, iid, true_r, est, {}))

print(f"총 {len(final_fused_predictions)}개의 융합 예측 점수 계산 완료.")


# 융합 추천 방식의 지표 계산 (모든 테스트 사용자에 대한 평균)
precision_fused_all, recall_fused_all, ndcg_fused_all = calculate_metrics(
    final_fused_predictions, testset_surprise, k=10
)

print(f"융합 추천 Precision@10 (평균): {precision_fused_all:.4f}")
print(f"융합 추천 Recall@10 (평균): {recall_fused_all:.4f}")
print(f"융합 추천 NDCG@10 (평균): {ndcg_fused_all:.4f}")


--- 기본 SVD 모델 (예측 평점만) Top-N 추천 지표 ---
기본 SVD Precision@10: 0.2528
기본 SVD Recall@10: 0.9777
기본 SVD NDCG@10: 0.9523

--- 융합 추천 모델 (예측 + 감성 유사도) Top-N 추천 지표 ---
모든 테스트 사용자-아이템 쌍에 대한 융합 점수 계산 중 (시간이 다소 소요될 수 있습니다)...
총 85791개의 융합 예측 점수 계산 완료.
융합 추천 Precision@10 (평균): 0.2530
융합 추천 Recall@10 (평균): 0.9779
융합 추천 NDCG@10 (평균): 0.9533


In [22]:
from surprise import KNNBasic

print("--- 협업 필터링 (UBCF, IBCF) 모델 학습 및 평가 시작 ---")

# --- UBCF (User-Based Collaborative Filtering) ---
print("\n--- UBCF (이웃 수 100) 모델 학습 및 평가 ---")
# user_based=True로 설정하여 UBCF 사용
ubcf_algo = KNNBasic(k=100, sim_options={'name': 'cosine', 'user_based': True}, random_state=42)
start_time = time.time()
ubcf_algo.fit(trainset_surprise)
end_time = time.time()
print(f"UBCF 모델 학습 시간: {end_time - start_time:.2f}초")

predictions_ubcf = ubcf_algo.test(testset_surprise)
precision_ubcf, recall_ubcf, ndcg_ubcf = calculate_metrics(predictions_ubcf, testset_surprise, k=10)

print(f"UBCF Precision@10: {precision_ubcf:.4f}")
print(f"UBCF Recall@10: {recall_ubcf:.4f}")
print(f"UBCF NDCG@10: {ndcg_ubcf:.4f}")

# --- IBCF (Item-Based Collaborative Filtering) ---
print("\n--- IBCF (이웃 수 100) 모델 학습 및 평가 ---")
# user_based=False로 설정하여 IBCF 사용
ibcf_algo = KNNBasic(k=100, sim_options={'name': 'cosine', 'user_based': False}, random_state=42)
start_time = time.time()
ibcf_algo.fit(trainset_surprise)
end_time = time.time()
print(f"IBCF 모델 학습 시간: {end_time - start_time:.2f}초")

predictions_ibcf = ibcf_algo.test(testset_surprise)
precision_ibcf, recall_ibcf, ndcg_ibcf = calculate_metrics(predictions_ibcf, testset_surprise, k=10)

print(f"IBCF Precision@10: {precision_ibcf:.4f}")
print(f"IBCF Recall@10: {recall_ibcf:.4f}")
print(f"IBCF NDCG@10: {ndcg_ibcf:.4f}")

# --- SVD (잠재 요인 1개) 모델 재확인 ---
print("\n--- SVD (잠재 요인 1개) 모델 재학습 및 평가 (비교용) ---")
# SVD 모델의 n_factors를 1로 설정
svd_l1_algo = SVD(n_factors=1, n_epochs=50, lr_all=0.01, reg_all=0.05, random_state=42) # 파라미터는 예시
start_time = time.time()
svd_l1_algo.fit(trainset_surprise)
end_time = time.time()
print(f"SVD (L1) 모델 학습 시간: {end_time - start_time:.2f}초")

predictions_svd_l1 = svd_l1_algo.test(testset_surprise)
precision_svd_l1, recall_svd_l1, ndcg_svd_l1 = calculate_metrics(predictions_svd_l1, testset_surprise, k=10)

print(f"SVD (L1) Precision@10: {precision_svd_l1:.4f}")
print(f"SVD (L1) Recall@10: {recall_svd_l1:.4f}")
print(f"SVD (L1) NDCG@10: {ndcg_svd_l1:.4f}")

--- 협업 필터링 (UBCF, IBCF) 모델 학습 및 평가 시작 ---

--- UBCF (이웃 수 100) 모델 학습 및 평가 ---
Computing the cosine similarity matrix...
Done computing similarity matrix.
UBCF 모델 학습 시간: 27.06초
UBCF Precision@10: 0.2534
UBCF Recall@10: 0.9783
UBCF NDCG@10: 0.9510

--- IBCF (이웃 수 100) 모델 학습 및 평가 ---
Computing the cosine similarity matrix...
Done computing similarity matrix.
IBCF 모델 학습 시간: 1.75초
IBCF Precision@10: 0.2478
IBCF Recall@10: 0.9735
IBCF NDCG@10: 0.9202

--- SVD (잠재 요인 1개) 모델 재학습 및 평가 (비교용) ---
SVD (L1) 모델 학습 시간: 4.06초
SVD (L1) Precision@10: 0.2531
SVD (L1) Recall@10: 0.9780
SVD (L1) NDCG@10: 0.9522
